# van Emde Boas Tree

## Why do we need van Emde Boas Tree?

We know about many datastructures that support priority queue operations
eg binary heaps, Red Black Trees, fibonacci heaps.

But in each of these atleast one among INSERT and EXTRACT-MIN takes $\Omega(logn)$ time.
A verification of the above conclusion is, because all of the above data structures are based on comparison between the keys and the lower bound for comparison based sorting is $\Omega(nlogn)$  
if we would have been able to perform INSERT and EXTRACT-MIN in $o(logn)$ then we could 
sort n keys in o(nlogn) using heap sort (n INSERTions followed by n EXTRACT-MIN operations).

But we also know that non-comparison based sorting techniques break this lower bound for sorting eg: Counting Sort takes $O(n+k)$ time to sort n elements in the range 0 to k.
Of course it is only true when integers(keys) are in a bounded range.

So can we improve priority queue operations when keys are in a bounded range?  
van Emde Boas Tree uses this idea to support priority-queue operations in $O(log log u)$ time
when all keys belong to the range from 0 to u-1.

## Operations  

van Emde Boas Tree supports the following operations:- 
  
`SEARCH(S,x)` : Search for x in the set S. Returns `True` if x exists in S, `False` otherwise.

`INSERT(S,x)` : Inserts element x into S.

`DELETE(S,x)` : Deletes element x from S, assuming that S contains x. 

`MINIMUM(S)` : Returns the minimum element in the set, None if S is empty. 

`MAXIMUM(S)` : Returns the minimum element in the set, None if S is empty. 

`SUCCESSOR(S,x)` : Returns the smallest element greater that x, `None` if x = `MAXIMUM(S)`

`PREDECESSOR(S,x)` : Returns the largest element smaller than x, `None` if x = `MINIMUM(S)` 

All these operations except MINIMUM and MAXIMUM run in $O(loglogu)$.  
MINIMUM and MAXIMUM are O(1) time operations.

## Real world applications 
- Routing packets to a subnet.
![alt text](img/router.jpeg)

Problem in routing packets :-


We know that the IP adresses are distributed in blocks where each block is a subnet. Each subnet has some range, for example assume Router C shown above has one of its ports corresponding to subnet A which has IP addresses in the range $[ 128.23.45.0 - 128.255.255.255 ]$. 

Whenever a packet arriving at the router has a destination address falling in above range, this packet has to be forwarded to the port corresponding to subnet A.Eg a packet with destination IP address 128.28.67.90 arrives at Router C ,since it falls in the range of subnet A, so it will be forwarded to port corresponding to the subnet A.  
Each router in the network may have multiple ports , and each port is used to forward the packet to a particular subnet.  
Every time a packet arrives at the router,using subnet mask its outgoing port number is calculated ,routing table dictates the port to which the packet has to be forwarded, this takes lot of time as millions of packets are processed in the network and the processing delay includes, using masking tool to get the network id and then looking up into the routing table ,although binary search trees come to rescue but still we can get rid of $Olog(n)$ by using van Emde Boas trees in $O(log log u)$ time.

Remedy :-


Now starting IP address of each range(subnet) is a node and we are required to find the predecessor of the IP address of the arriving packet to forward the packet to a port efficiently.

Eg. a packet with destination IP address 128.28.67.90 arrives at Router C ,prdecessor of this address is 128.23.45.0 which is the starting address of the subnet A, so it will be forwarded to port corresponding to the subnet A.vEB-tree makes the search exponentially faster than the BST.Successor and predecessor operations will take $O(loglogu)$ time where as balanced binary search tree takes $O(logn)$ time.  
 
In general van Emde Boas trees can be used anywhere in place of a normal binary search tree as long as the keys in the search tree are integers in some fixed range. Thus for applications where we are required to find the integer in a set that is closest to some other integer(predecessor or successor), using a vEB-tree can potentially be faster than using a simple balanced binary search tree.

As an example, of you have a linear layout of stores on some line and want to find the closest store to some particular customer, using a vEB-tree could make the search exponentially faster than the BST.

## Notations:  
T = van Emde Boas Tree  
n =  number of elements in vEB Tree  
u = range of elements \[0,u-1\]  
i.e., $ \forall x \in\mathbb T $;   $0<=x<u$  
For simplicity we assume that u is always an exact power of 2 i.e., $u=2^k$ where $k \in\mathbb Z^+$

## Background
Before using a van Emde Boas Tree we need to initialize the whole empty tree. For a universe of size u, space requirement of a vEB Tree is O(u) and creating such an empty tree takes O(u) time. Whereas creating an empty Red Black Tree takes constant amount of time. Therefore it would be a very bad idea to use vEB Tree when we need to perform only a small number of operations. Time spent in creating the datastructure would exceed the time spent on performing operations.

The following recurrence relation characterises the running time of various operations of vEB Tree  
T(u) = T($\sqrt{u}$) + O(1)  


Let m = log u,so that u = 2$^m$
now we have  
T(2$^m$) = T(2$^\frac{m}{2}$) + O(1)

now we rename T(2$^m$) to S(m)    
S(m) = S($\frac{m}{2}$) + O(1).

By case 2 of the master method, this recurrence has the solution  
S(m) = O(log m).  
Moving back from S(m) to T(u)  
S(m) = O(log m)  
=> T(2$^m$) =  O(log m)  
Replacing m = log u   
=> T(u) = O(log log u) 

## vEB Tree Structure
As clear from the recurrence relation above, we are dividing the universe size u to $\sqrt{u}$ recursively, but what if u is not of the form $2^{2^k}$ because then $\sqrt {u}$ won't be an integer. We solve this problem as follows:-  
Since universe size can be of the form $2^k$ for some  $k \in\mathbb Z^+$, and so u can be an odd power of 2 so  we divide the lg u bits of a number into the most significant $\displaystyle \left \lceil \frac{\lg u}{2} \right \rceil$ bits and the least significant $\displaystyle \left \lfloor \frac{\lg u}{2} \right \rfloor$ bits.  

### Implementation
Implementation wise we need to define two separate square roots.
We define upper root as $2 ^{ceil{(log(u) / 2)}}$ and lower root as $2 ^{floor{(log(u) / 2)}}$
```python
upper_root = 2 ** ceil(log2(u) / 2)  # eg 4
lower_root = 2 ** floor(log2(u) / 2)  # eg 2

class VEBTree:
    def __init__(self, u):
        self.u = u  # eg 8
        self.min = None
        self.max = None

        if u > 2:
            # unless u equals base size 2,
            # attribute summary points to a veb tree of size upper_root
            # and each cluster in cluster [0 ... upper_root -1 ] points to vEB Trees of size lower_root.
            # eg u = 8, upper_root = 4, lower_root = 2
            # summary point to a veb tree of size 4
            # cluster is an array of size 4
            # each element of cluster points to a veb tree of size 2
            # so cluster points to 4 veb(2) tree
            self.summary = VEBTree(upper_root)
            self.cluster = [VEBTree(lower_root) for _ in range(upper_root)]
```

>A vEB Tree structure contains u (the universe size), min (minimum value in the tree root at that structure) , max (maximum value rooted at that structure).  
In addition to these a non base case vEB structure also contains a summary pointer and an array of cluster pointers. Summary list contains a summary about which clusters contain atleast one key clusters themselves are vEB Trees of size $\sqrt{u}$.

> From the above implementation we see that the following recurrence relation characterizes the space requirement $S(u)$ of a van Emde Boas tree with universe size u:-  
$S(u) = (\sqrt{u} + 1) S(\sqrt{u}) + \Theta(\sqrt{u})$  
$1$: summary cluster of size $S(\sqrt{u})$.  
$\sqrt{u}$: sub clusters each of size $S(\sqrt{u})$.  
$\theta(\sqrt{u})$: pointers of clusters.  
Solving this recurrence gives $S(u) = O(u)$

## Finding minimum or maximum element in the set
### Algorithm
Finding minimum or maximum element is as simple as returning the stored minimum or maximum value in the cluster.  
Clearly both `MINIMUM` and `MAXIMUM` are constant time operations because the min and max values are cached in the vEB structure.

### Implementation
```python
def MINIMUM(V):
    return V.min


def MAXIMUM(V):
    return V.max
```

## Insertion into vEB Tree
### Implementation
```python
def INSERT_EMPTY(V, x):
    V.min = V.max = x


def INSERT(V, x):
    # V is an empty vEB Tree (Base case)
    if V.min is None:
        INSERT_EMPTY(V, x)
        return

    # else V is non empty
    else:
        if x < V.min:
            # If x < min, then x needs to become the new min.
            # But we don't want to lose the original min.
            # So we need to insert it into one of V's clusters.

            # exchange x and V.min
            x, V.min = V.min, x
            # now insert the original min (now x) into one of the V's clusters

        if V.u > 2:
            # Non base case

            # check whether the cluster that x will go into is currently empty
            # checking MINIMUM or MAXIMUM is sufficient to check for empty cluster
            if MINIMUM(V.cluster[V.high(x)]) is None:
                # insert x's cluster number into summary
                INSERT(V.summary, V.high(x))
                # insert x into the empty cluster
                INSERT_EMPTY(V.cluster[V.high(x)], V.low(x))

            else:
                # x's cluster is not empty
                # so we do not need to update the summary, since x's cluster number is already a member of the summary.

                # insert x into its cluster
                INSERT(V.cluster[V.high(x)], V.low(x))

        # update max
        if x > V.max:
            V.max = x
```

## Deletion from vEB Tree
### Implementation
```python
# assumes that x is currently an element in the set
# represented by the vEB tree V.
# this means if the tree contains only one key
# then no matter what value of x you pass to delete,
# the existing tree element will be deleted
def DELETE(V, x):
    # exactly one element in the tree
    if V.min == V.max:
        V.min = V.max = None

    # Base case: set min and max to the one remaining element.
    elif V.u == 2:
        # if exactly 2 elements exist
        if x == 0:  # and the key to be deleted is 0
            # then set min and max to key 1
            V.max = V.min = 1
        else:  # else key to be deleted is 1
            # so set min and max to key 0
            V.max = V.min = 0

    else:
        # we will have to delete an element from a cluster

        if x == V.min:  # we need to delete the min element
            # but before that we need to find the new min which is some other element within one of V's clusters

            # first_cluster = the cluster id that contains the lowest element other than min
            first_cluster = MINIMUM(V.summary)  # cluster id of cluster containing new min

            # x = lowest element in the found cluster
            x = V.index(first_cluster, MINIMUM(V.cluster[first_cluster]))

            # x becomes new min
            V.min = x

            # now x will be deleted from its cluster

        # Now we need to delete element x from its cluster,
        # whether x was the value originally passed to DELETE()
        # or x is the element becoming the new minimum.
        DELETE(V.cluster[V.high(x)], V.low(x))  # delete x from its cluster

        # That cluster might now become empty
        if MINIMUM(V.cluster[V.high(x)]) is None:
            # if it does, then we need to remove x's cluster number from the summary,
            DELETE(V.summary, V.high(x))

            # After updating the summary, we might need to update max if x is max
            if x == V.max:  # check whether we are deleting max element of V

                # summary_max = the number of the highest numbered nonempty cluster.
                # This works bcoz we have already recursively called DELETE on V.summary
                # and so V.summary.max has already been updated.
                summary_max = MAXIMUM(V.summary)

                if summary_max is None:
                    # If all of V's clusters are empty, then the only remaining element in V is min
                    V.max = V.min  # update max accordingly

                else:
                    # else set max to the maximum element in the highest numbered cluster
                    V.max = V.index(summary_max, MAXIMUM(V.cluster[summary_max]))

        # else if the cluster did not become empty (there is at least one element in x's cluster even after deleting x.)
        # then although we do not have to update the summary in this case, we might have to update max.
        elif x == V.max:  # if max element was deleted, update max
            V.max = V.index(V.high(x), MAXIMUM(V.cluster[V.high(x)]))

```

> For full source code of vEB Tree implementation see [src/kruskal_impl/datastructures/vEB_Tree.py](src/kruskal_impl/datastructures/vEB_Tree.py) 

# Fibonacci heap

|Procedure   |   Binary heap (worst-case)|   Fibonacci heap(amortized)|
|------------|---------------------------|----------------------------|
|   MAKE-HEAP|  $\Theta(1)$ |$\Theta(1)$   |
|   INSERT|   $\Theta(\lg {n})$|$\Theta(1)$   |
|   MINIMUM|   $\Theta(1)$|   $\Theta(1)$|
|   EXTRACT-MIN|   $\Theta(\lg {n})$|$O(\lg {n})$   |
|   UNION|   $\Theta(n)$|$\Theta(1)$   |
|   DECREASE-KEY|   $\Theta(\lg {n})$| $\Theta(1)$  |
|   DELETE|   $\Theta(\lg {n})$|$O(\lg {n})$   |

Fibonacci heap is a data structure for priority queue operations, consisting of a collection of heap-ordered 
trees. It has a better amortized running time than many other priority queue data structures including the binary heap and binomial heap. Fibonacci heaps are named after the Fibonacci numbers, which are used in their running time analysis.


For the Fibonacci heap, the find-minimum operation takes constant $O(1)$ amortized time.The insert and decrease key operations also work in constant amortized time. Deleting an element (most often used in the special case of deleting the minimum element) works in O(log n) amortized time,where n is the size of the heap.

This means that starting from an empty data structure, any sequence of an insert and decrease key operations and b delete operations would take $O(a + b log n)$ worst case time, where n is the maximum heap size.

Below are some interesting facts about Fibonacci Heap

The reduced time complexity of Decrease-Key has importance in Dijkstra and Prim algorithms. With Binary Heap, time complexity of these algorithms is O(VLogV + ELogV). If Fibonacci Heap is used, then time complexity is improved to O(VLogV + E)
Although Fibonacci Heap looks promising time complexity wise, it has been found slow in practice as hidden constants are high (Source Wiki).
Fibonacci heap are mainly called so because Fibonacci numbers are used in the running time analysis. Also, every node in Fibonacci Heap has degree at most O(log n) and the size of a subtree rooted in a node of degree k is at least Fk+2, where Fk is the kth Fibonacci number.

# Kruskal's minimum-spanning-tree algorithm

![alt text](img/Kruskalimg.png)

Kruskal's algorithm builds the minimum spanning tree in forest. Initially, each vertex is in its own tree within the forest as a single vertex is also a tree. Then, algorithm considers each edge in non increasing order of the edge weights.  
If an edge (u, v) connects two different trees, then (u, v) is added to the set of edges of the MST, and two trees connected by an edge (u, v) are merged 
into a single tree, on the other hand, if an edge (u, v) connects two vertices in the same tree,
then edge (u, v) is discarded.

Kruskal's algorithm requires a subroutine call to a method, to check wether adding an edge forms a cycle in the graph. Cycle in a static graph can be detected using Depth First Search in $O(V+E)$.Since the edges are added dynamically so it is more efficient to use disjoint-set data structure to ensure that adding an edge does not form a cycle rather than running an $O(V+E)$ algorithm each time an edge is added.

This algorithm also uses a min-heap data structure to get the edges in $O(log v)$


Operations supported by disjoint-set data structure :-

Make_SET(v):  Creates a new set whose only member is pointed to by v. Note that for this operation v must already be in a set.

FIND_SET(v):  Returns a pointer to the set containing v.

UNION(u, v):  Unites the dynamic sets that contain u and v into a new set that is union of these two sets.

Using both path compression, and union by rank or size ensures that the amortized time per operation is only $O( \alpha{(n)})$  which is optimal, where  $\alpha{(n)}$ is the inverse Ackermann function. This function has a value $\alpha{(n)}$ <5 for any value of n that can be written in this physical universe, so the disjoint-set operations take place in essentially constant time.



Algorithm :-

Start with an empty set A, and select at every stage the shortest edge that has not been chosen or rejected, regardless of where this edge is situated in the graph.

*KRUSKAL*(G):  
A = NULL                           #Initially the spanning tree is empty.
For each vertex v ∈ G.V:  
MAKE-SET(v)                        #Initialises the disjoint-set data structure
For each edge (u, v) ∈ G.E         #edges are extracted from the min heap
    if FIND-SET(u) ≠ FIND-SET(v):  #Ensures that the edge does not form a cycle.      
    A = A ∪ {(u, v)}               #Adds the edge to our spanning tree if edge does not form a cycle.
    UNION(u, v)                    #Updates the Disjoint-set data structure
return A                           #returns the final spanning tree



Overall time complexity of Kruskal's MST algorithm :-
1. Build heap takes $O(E)$, for E edges.
2. Disjoint-set data structure takes $O(m \alpha(n))$ for m operations, m is typically the number of edges,and alpha is a slowly growing function.
3. Each extract-min opertaion takes $O(log E)$
Total :-
$O(E)$ + $O(m)$ + $O(E log E)$
finally it is $O(E log E)$

# Implement kruskal using vEB Tree
We can implement kruskal by using edge weights as the keys of the vEB Tree. But before using a vEB Tree we need to initialize the tree with the universe size u, so we need to know the maximum possible edge weight in our graph over which kruskal MST algorithm is to be applied.  
For simplicity let us assume that the edge weights in the graph will never be more than **1023**.  
So initializing vEB Tree with u = 1024 solves this issue.

When implementing kruskal using vEB Tree we need 2 modifications to our original implementation of vEB Tree.  
1. Support for duplicate keys (because there can be multiple edges with same edge weight).
2. Store satellite data in addition to the keys (so that we could detect a cycle).

# Modified vEB Tree implementation
1. In order to add support for duplicate keys we store a count of min and max which specify how many times a key(edge weight) appears.
2. To store satellite data in addition to the keys we maintain a list of objects which stores the satellite object for each key. We need a list because a key can now be repeated, so same key may have more than one satellite data (two or more edges with same weight).

### Modified vEB Structure
```python
class VEBTree:
    def __init__(self, u):
        ...
        
        # Counters to maintain number of times a key appears
        self.min_cnt = 0
        self.max_cnt = 0
        
        # A list of objects for each key.
        self.min_data = None
        self.max_data = None

        if u > 2:
            ...
```

We will also need to modify our `INSERT` and `DELETE` implementations to fulfill above two requirements.
> We use the python's copy.copy() / copy.deepcopy() module to create a copy the original list of satellite objects, this is needed so that we do not accidentally modify the same list object from different places.
```python
from copy import copy
```


### Modified INSERT implementation
```python
# our insert functions now accept two additonal parameters: satellite data and key count

def INSERT_EMPTY(V, x, data, n):
    ...
    V.min_cnt = V.max_cnt = n
    V.min_data = copy(data)
    V.max_data = copy(data)


def INSERT(V, x, data, n=1):
    if V.min is None:
        INSERT_EMPTY(V, x, data, n)
        return

    if x == V.max:
        V.max_cnt += n
        V.max_data.extend(data)
    if x == V.min:
        V.min_cnt += n
        V.min_data.extend(data)
        return

    if x < V.min:
        ...
        n, V.min_cnt = V.min_cnt, n
        data, V.min_data = copy(V.min_data), copy(data)

    if V.u > 2:
        if MINIMUM(V.cluster[V.high(x)]) is None:
            ...

        else:
            ...

    if x > V.max:
        ...
        V.max_cnt = n
        V.max_data = copy(data)
```
> view full source at [src/kruskal_impl/datastructures/vEB_with_dups_n_satellite.py](src/kruskal_impl/datastructures/vEB_with_dups_n_satellite.py) 


### Modified DELETE implementation
> view full source at [src/kruskal_impl/datastructures/vEB_with_dups_n_satellite.py](src/kruskal_impl/datastructures/vEB_with_dups_n_satellite.py) 

In addition to above changes we also add a `GET_SATELLITE` helper method which returns the list of satellite objects stored with a given key.
```python
def GET_SATELLITE(V, x):
    if x == V.min:
        return V.min_data
    if x == V.max:
        return V.max_data
    if V.u == 2:
        return None

    return GET_SATELLITE(V.cluster[V.high(x)], V.low(x))
```

# Kruskal implementations API
Kruskal implementations have the following API:-
```
Graph(V): Initialize the graph with V vertices.  
Graph.insert_edge(u,v,w): Insert an edge with weight w incident on edges u and v to the graph.
Graph.kruskal(show_mst=False): Runs Krukal's MST algorithm on the graph. Accepts an optional parameter show_mst which is a boolean.
If show_mst is True then generated minimum spanning tree is printed to the console.
Graph.kruskal() returns the cost of minimum spanning tree 
```


**source code links:**  
 [kruskal using list](src/kruskal_impl/kruskal_list_sort.py)  
 [kruskal using vEB Tree](src/kruskal_impl/kruskal_veb.py)  
 [kruskal using fibonacci heap](src/kruskal_impl/kruskal_fib_heap.py) 

# Comparison between the 3 implementations
1. Kruskal implemented by storing edges in a list and sorting the list.
    - Initializing edge list takes $O(1)$ (initializes an empty list \[\])
    - Insertion of an edge takes $O(1)$ amortized time.
    - Sorting takes $O(n\lg n)$ time.
    - Extracting the edge with minimum weight takes $O(1)$ time (since we do not actually delete the element, instead we update a pointer to point to the next minimum edge)
    - Source code: [src/kruskal_impl/kruskal_list_sort.py](src/kruskal_impl/kruskal_list_sort.py)


2. Kruskal implemented using vEB Tree
    - Initializing edge list takes constant time but the constant factor is very high because u = 1024 so a vEB Tree of size $O(u)$ is created.
    - Inserting an edge takes $O(\lg\lg u)$ time.
    - Extracting the minimum weighted edge (call to MINIMUM and DELETE) takes a total of $O(\lg\lg u)$ time.
    - Source code: [src/kruskal_impl/kruskal_veb.py](src/kruskal_impl/kruskal_veb.py)


3. Kruskal implemented using Fibonacci Heap
    - Initializing edge list takes $O(1)$ time.
    - Inserting an edge takes $\Theta(1)$ amortized time.
    - Extracting the minimum weighted edge takes $O(\lg n)$ time.
    - Source code: [src/kruskal_impl/kruskal_fib_heap.py](src/kruskal_impl/kruskal_fib_heap.py)
    

## Comparison graph
![alt text](img/comparison.png)


### Explanation
As we can see, Fibonacci heap implementation outperforms the vEB Tree implementation when number of edges is small (less than ~1500 edges). This is because creating an empty vEB Tree has a high overhead which does not make any sense when number of operations (edges in our graph) is less. Whereas when the input size increases beyond a certain value, the vEB Tree implementation performs better than the fibonacci heap implementation. In the above graph the size of universe of vEB Tree is fixed to 1024, so when there are many edges >>1024, then a large number of edges have the same edge weight, so they grow the underlying list associated with that edge.   

## How to run kruskal implementations?
All three implementations of kruskal take input from standard input in the following format:-  
The first line contains 2 integers, V and E  
V = The number of vertices of graph   
E = the number of edges of the graph  
Next E lines contains edges in the format: u v w  
where w is the weight of the edge incident on the vertices u and v  
0 <= u,v <=V-1  

Sample input:-  
3 3  
1 0 67  
2 0 46  
2 1 75  

Sample output:-  
113  


## How to use these implementations? 
A typical usage for these implementations would be:-
```python
import kruskal_veb as kruskal_veb

g = kruskal_veb.Graph(V)
    for edge in edges:
        u, v, w = edge
        g.insert_edge(u, v, w)

    g.kruskal()
```

**Extra parameter for Kruskal based on vEB Implementation** :-  
[src/kruskal_impl/kruskal_veb.py](src/kruskal_impl/kruskal_veb.py)'s Graph constructor takes one extra parameter in addition to V, it also takes UNIVERSE_SIZE as input.  
**`UNIVERSE_SIZE`** is an optional parameter which defaults to *1024*.  

>When running [src/kruskal_impl/kruskal_veb.py](src/kruskal_impl/kruskal_veb.py) on custom input, make sure to set the universe size u correctly, it should be strictly greater than the maximum edge weight possible and must be a power of 2.  
A typical initialization would be as follows :-
```
    g = Graph(V, UNIVERSE_SIZE=2 ** math.ceil(math.log2(maximum_edge_weight)))
```

### How to get comparison graph
The graph above was generated by [src/kruskal_impl/kruskal_veb.py](src/kruskal_impl/kruskal_veb.py) file. This program reads test input files from [tests/input](tests/input) directory and uses python's [timeit](https://docs.python.org/3/library/timeit.html) module to record the execution time of the implementations. It then uses [matplotlib](https://matplotlib.org/) to plot the curves.

> You can also run the program on sample test cases supplied. Sample tests can be found in tests directory.


### How to generate test cases?
You can use [src/utils/test_generator.py](src/utils/test_generator.py) to generate test case files in the format specified above. The generated file will be placed in the current working directory.  